# Nettoyage et Filtrage du Dataset
Ce notebook effectue les premières étapes de traitement sur le fichier CSV original :
- Suppression des lignes vides
- Suppression des colonnes inutiles
- Suppression des doublons
- Filtrage des stations de métro
- Séparation en train/test


In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Charger le dataset brut
df = pd.read_csv('../data/raw/qualite-de-lair-dans-le-reseau-de-transport-francilien.csv', sep=';', low_memory=False)
df.head()

,Identifiant station,Nom de la Station,Nom de la ligne,Niveau de pollution aux particules,Niveau de pollution,Incertitude,Recommandation de surveillance,Action(s) QAI en cours,Lien vers les mesures en direct,Durée des mesures,Mesures d’amélioration mises en place ou prévues,stop_lon,stop_lat,point_geo,pollution_air,air,niveau,actions,niveau_pollution
0,IDFM:490917,Aéroport d'Orly,Métro 14,pas de données,pas de données,Évaluation : impossible faute d’informations,non,NaN,NaN,NaN,NaN,2.362143,48.727911,"48.72791074498099, 2.3621434233506027",pas de données,NaN,Pas de données,NaN,pas de données
1,IDFM:21994,Arts et Métiers,Métro 3,pollution faible,FAIBLE,Évaluation : modélisation reposant notamment s...,non,NaN,NaN,NaN,NaN,2.356723,48.865535,"48.86553459885931, 2.3567233472754983",pollution faible - incertitude forte,NaN,Pollution faible,NaN,pollution faible
2,IDFM:monomodalStopPlace:46395,Athis-Mons,RER C,station aérienne,station aérienne,NaN,non,NaN,https://www.airparif.fr/,NaN,NaN,2.403669,48.712668,"48.71266832171659, 2.403668724287269",station aérienne,NaN,Station aérienne,NaN,station aérienne
3,IDFM:monomodalStopPlace:43071,Aulnay-sous-Bois,RER B,station aérienne,station aérienne,NaN,non,NaN,https://www.airparif.fr/,NaN,NaN,2.495597,48.932151,"48.932150935734064, 2.495597257086315",station aérienne,NaN,Station aérienne,NaN,station aérienne
4,IDFM:monomodalStopPlace:45447,Avenue Henri Martin,RER C,pollution faible,FAIBLE,Évaluation : campagne de mesure ponctuelle,non,NaN,NaN,campagne de mesure ponctuelle,NaN,2.272235,48.865455,"48.86545537297977, 2.2722351438875577",pollution faible - incertitude faible,NaN,Pollution faible,NaN,pollution faible


In [2]:
# Supprimer les lignes vides et les doublons
df.dropna(how='all', inplace=True)
df.drop_duplicates(inplace=True)
df.reset_index(drop=True, inplace=True)
df.shape

(662, 19)

In [3]:
# Afficher les colonnes
df.columns.tolist()

['Identifiant station',
 'Nom de la Station',
 'Nom de la ligne',
 'Niveau de pollution aux particules',
 'Niveau de pollution',
 'Incertitude',
 'Recommandation de surveillance',
 'Action(s) QAI en cours',
 'Lien vers les mesures en direct',
 'Durée des mesures',
 'Mesures d’amélioration mises en place ou prévues',
 'stop_lon',
 'stop_lat',
 'point_geo',
 'pollution_air',
 'air',
 'niveau',
 'actions',
 'niveau_pollution']

In [4]:
# Supprimer les colonnes jugées inutiles
colonnes_a_supprimer = [
    'Lien vers les mesures en direct', 'Durée des mesures', 'Mesures d’amélioration mises en place ou prévues',
    'point_geo', 'pollution_air', 'niveau', 'actions'
]
df.drop(columns=colonnes_a_supprimer, inplace=True, errors='ignore')
df.head()

,Identifiant station,Nom de la Station,Nom de la ligne,Niveau de pollution aux particules,Niveau de pollution,Incertitude,Recommandation de surveillance,Action(s) QAI en cours,stop_lon,stop_lat,air,niveau_pollution
0,IDFM:490917,Aéroport d'Orly,Métro 14,pas de données,pas de données,Évaluation : impossible faute d’informations,non,NaN,2.362143,48.727911,NaN,pas de données
1,IDFM:21994,Arts et Métiers,Métro 3,pollution faible,FAIBLE,Évaluation : modélisation reposant notamment s...,non,NaN,2.356723,48.865535,NaN,pollution faible
2,IDFM:monomodalStopPlace:46395,Athis-Mons,RER C,station aérienne,station aérienne,NaN,non,NaN,2.403669,48.712668,NaN,station aérienne
3,IDFM:monomodalStopPlace:43071,Aulnay-sous-Bois,RER B,station aérienne,station aérienne,NaN,non,NaN,2.495597,48.932151,NaN,station aérienne
4,IDFM:monomodalStopPlace:45447,Avenue Henri Martin,RER C,pollution faible,FAIBLE,Évaluation : campagne de mesure ponctuelle,non,NaN,2.272235,48.865455,NaN,pollution faible


In [5]:
# Garder uniquement les stations de métro
df_metro = df[df['Nom de la ligne'].str.contains('Métro', na=False)]
df_metro.shape

(404, 12)

In [6]:
# Séparer en train (70%) et test (30%)
train, test = train_test_split(df_metro, test_size=0.3, random_state=42)

# Sauvegarder les fichiers
train.to_csv('../data/processed/train.csv', index=False)
test.to_csv('../data/processed/test.csv', index=False)
print('✅ Fichiers train.csv et test.csv enregistrés.')

✅ Fichiers train.csv et test.csv enregistrés.
